In [ ]:
#Installing the required libraries
!pip install langchain==0.0.266
!pip install openai==0.27.9
!pip install pandas==1.5.3
!pip install langchain_experimental==0.0.11

In [24]:
import pandas as pd
import sqlite3
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [25]:
# File path of the CSV file
file_path = '/Users/arun/Development/generativeai/3.dbbot/3idiots.csv'

#creating database

# SQLite database file path
database_path = 'database.db'

def convert_csv_db(csv_path, db_path):

    # Create a connection to the SQLite database
    connection = sqlite3.connect(db_path)


    # Read the CSV files into pandas DataFrames
    movie_df = pd.read_csv(csv_path)

    # Execute SQL CREATE TABLE statement to create the movie table

    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS Movie (Actors text, Directed text , Produced text , Cinematographer text , Release_Date text);
    '''

    # Execute the CREATE TABLE statement
    connection.execute(create_table_sql)

    # Insert the data from DataFrames into the table
    movie_df.to_sql('3idiots', connection, if_exists='append', index=False)

    # Commit the changes and close the connection
    connection.commit()
    connection.close()

    return

convert_csv_db(file_path, database_path)

In [30]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY' #read the API-key from local environment

In [26]:
movie_db = SQLDatabase.from_uri('sqlite:///database.db')
english_sql_llm = OpenAI(temperature=0) # here the temperature is set to zero as we do not need any variability
movie_sql_chain = SQLDatabaseChain.from_llm(llm = english_sql_llm,
                                     db = movie_db,
                                     verbose=True)
                                     #create a langchain with one node(english_sql_llm)

In [28]:
movie_sql_chain.run("Who are the actors in this movie")



> Entering new SQLDatabaseChain chain...
Who are the actors in this movie
SQLQuery:SELECT "Actors" FROM "3idiots" LIMIT 5;
SQLResult: [('Aamir Khan',), ('R Madhavan',), ('Sharman Joshi',), ('Kareena Kapoor',), ('Boman Irani',)]
Answer:The actors in this movie are Aamir Khan, R Madhavan, Sharman Joshi, Kareena Kapoor, and Boman Irani.
> Finished chain.


'The actors in this movie are Aamir Khan, R Madhavan, Sharman Joshi, Kareena Kapoor, and Boman Irani.'

In [29]:
movie_sql_chain.run("When was the movie released?")



> Entering new SQLDatabaseChain chain...
When was the movie released?
SQLQuery:SELECT "Release_Date" FROM "3idiots" LIMIT 5;
SQLResult: [('25-Dec-09',), (None,), (None,), (None,), (None,)]
Answer:The movie was released on 25-Dec-09.
> Finished chain.


'The movie was released on 25-Dec-09.'